# Modelo de Robots inteligentes que recogen cajas

### Imports

In [ ]:
  !pip3 install mesa matplotlib numpy pandas
from mesa import Agent, Model # Import the Agent and Model classes
from mesa.space import MultiGrid # MultiGrid is a grid where each cell can hold multiple obj (Va a contener 1 tipo piso y varias aspiradoras)
from mesa.time import SimultaneousActivation # SimultaneousActivation is a scheduler that activates all agents simultaneously, in random order, then moves on to the next step.
from mesa.datacollection import DataCollector # DataCollector is a class for collecting model-level and agent-level data.

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams["animation.embed_limit"] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Modelo

In [ ]:
estante_x = 0
estante_y = 0
lleno = None

def obtain_room(model):
    # Initialize the room as a grid of 0s width x height
    room = np.zeros((model.grid.width, model.grid.height))
    # For loop to obtain the values of each cell in the grid (floor)
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        # For loop to obtain the content of each cell in the grid (floor)
        for obj in cell_content:
            if isinstance(obj, Robot):
                # Change the value of the cell to 10 if there is a robot
                room[x][y] = 7
            else:  # If the cell has no robot then it is a floor
                if isinstance(obj, Floor):
                    # Change the value of the cell to 1 if there is a floor
                    room[x][y] = obj.state
                elif isinstance(obj, Wall):  # If the cell has no floor then it is a wall
                    room[x][y] = 10
    return room


class Robot(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.movements = 0
        self.state = 0
        self.new_position = None
        self.next_state = None

    def step(self):
        neighbors = self.model.grid.get_neighbors(self.pos, moore=False, include_center=True)
        global estante_x
        global estante_y
        global lleno
        for neighbor in neighbors:
            if isinstance(neighbor, Floor) and self.pos == neighbor.pos:
                neighbor.next_state = neighbor.state
                self.next_state = self.state
                if lleno == True:
                    if estante_y + 1 < self.model.cols:
                        estante_y += 1
                        lleno = False
                    else:
                        estante_y = 0
                        estante_x += 1
            
                # Si el vecino tiene caja y no es un estante y no esta cargando
                if neighbor.next_state == 1 and neighbor.pos != (estante_x, estante_y) and self.next_state == 0:
                    neighbor.next_state = neighbor.state - 1
                    self.next_state = self.next_state + 1
                
                if neighbor.pos == (estante_x, estante_y) and self.next_state == 1:
                    if neighbor.next_state < 5:
                        neighbor.next_state = neighbor.state + 1
                        self.model.boxes_stacked += 1
                        self.next_state = self.next_state - 1
                        if neighbor.next_state == 5:
                            lleno = True
                
                self.new_position = self.pos
                break
        
        if self.next_state == 1:
            if self.new_position != estante_y:
                if self.new_position[1] < estante_y:
                    self.new_position = (self.new_position[0], self.new_position[1] + 1)
                elif self.new_position[1] > estante_y:
                    self.new_position = (self.new_position[0], self.new_position[1] - 1)
            if self.new_position[0] != estante_x:
                self.new_position = (self.new_position[0] - 1, self.new_position[1])
        else:
            neighborhood = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)
            self.new_position = random.choice(neighborhood)
                    
                    
    def advance(self):
        neighbors = self.model.grid.get_neighbors(
            self.pos, moore=False, include_center=True
        )

        for neighbor in neighbors:
            if isinstance(neighbor, Floor) and self.pos == neighbor.pos:
                neighbor.state = neighbor.next_state
                
        cell_cont_in_new_pos = self.model.grid.get_cell_list_contents([self.new_position])
    
        num_robots_cell = len([obj for obj in cell_cont_in_new_pos  if isinstance(obj, Robot)])
    
        if num_robots_cell != 0:
            self.new_position = self.pos
        
        if  self.pos != self.new_position:
            self.movements += 1
            
        self.model.grid.move_agent(self, self.new_position)
        self.state = self.next_state


class Floor(Agent):
    def __init__(self, pos, model, state=0):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.state = state
        self.next_state = None


class Wall(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)


class Room(Model):
    def __init__(self, width, height, n_robots, n_boxes):
        self.n_robots = n_robots
        self.n_boxes = n_boxes
        self.schedule = SimultaneousActivation(self)
        self.grid = MultiGrid(width, height, True)
        self.boxes_stacked = 0
        self.cols = width
        self.rows = height
        self.collected_boxes = 0

        list_empty_cells = list(self.grid.empties)

        #list_empty_cells = list(self.grid.empties)
        # Initialize the Walls position
        # Start at the fourth row
        # for i in range(2, width-2, 3):
        #     for j in range(3, height-3):
        #         pos = (j, i)
        #         wall = Wall(pos, self)
        #         self.grid.place_agent(wall, pos)
        #         list_empty_cells.remove(pos)

        # Initialize the position of boxes
        #list_empty_cells = list(self.grid.empties)
        for i in range(n_boxes):
            # Choose a random position for the box
            pos = random.choice(list_empty_cells)
            # Create a box in that position
            box = Floor(pos, self, state=1)
            # Add the box to the grid
            self.grid.place_agent(box, pos)
            # Add the box to the schedule
            self.schedule.add(box)
            # Remove the position from the list of empty cells
            list_empty_cells.remove(pos)

        # Initialize the position of empty cells
        list_empty_cells = list(self.grid.empties)
        for cell in range(len(list_empty_cells)):
            pos = random.choice(list_empty_cells)
            # Create an empty cell
            piso = Floor(pos, self)
            # Add the empty cell to the grid
            self.grid.place_agent(piso, pos)
            # Add the empty cell to the schedule
            self.schedule.add(piso)
            list_empty_cells.remove(pos)

        # Initialize the position of robots
        for i in range(n_robots):
            celda = random.choice(random.choice(self.grid.grid))
            while(celda[0].state != 0 or len(celda) != 1):
                celda = random.choice(random.choice(self.grid.grid))
            robot = Robot(i, self)
            self.grid.place_agent(robot, celda[0].pos)
            self.schedule.add(robot)

        self.colectordatos = DataCollector(
            model_reporters={"Habitacion": obtain_room},
            agent_reporters={"Movimiento": lambda a: getattr(a, "movements", None)}
        )

    # Step function that is called at each step of the model
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()
        

    def allBoxesStacked(self):
        n_shelves = self.n_boxes // 5
        self.collected_boxes = 0
        for i in range(n_shelves):
            self.collected_boxes += self.grid.get_cell_list_contents([(0,i)])[0].state
        
        
        return self.collected_boxes == self.n_boxes
                

### Correr el modelo


In [ ]:
H = 10
W = 10

n_robots = 5

n_boxes = 40

MAX_TIME = float(input('Tiempo de ejecución (segundos): '))

start_time = time.time()

start_at_time = str(datetime.timedelta(seconds=MAX_TIME))

model = Room(W, H, n_robots, n_boxes)

while((time.time() - start_time) < MAX_TIME and not model.allBoxesStacked()):
    model.step()
    
exec_time = time.time() - start_time

#Imprimimos el tiempo de ejecucion
tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

Tiempo de ejecución (segundos): 1


### Visualización

In [ ]:
obtain_room = model.colectordatos.get_model_vars_dataframe()

In [ ]:
%%capture

# Plot the results
fig, axs = plt.subplots(figsize=(7, 7))
axs.set_xticks([])
axs.set_yticks([])

patch = plt.imshow(obtain_room.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(obtain_room.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(obtain_room) )

In [ ]:
anim

### Informe

In [ ]:
movimientos = model.colectordatos.get_agent_vars_dataframe()

print("Tiempo necesario hasta limpiar la habitacion: ", tiempo_ejecucion, "/", start_at_time )
print("Cantidad de cajas recogidas despues del termino de la simulación: ",  model.collected_boxes)
print("Numero de movimientos realizados por los agentes: ", movimientos.tail()['Movimiento'].sum())

Tiempo necesario hasta limpiar la habitacion:  0:00:00.102973 / 0:00:01
Cantidad de cajas recogidas despues del termino de la simulación:  40
Numero de movimientos realizados por los agentes:  1069.0


### ¿Como mejorar el rendimiento?

- La solución rapida y sencilla es jugar con las variables del entorno, tal como, el tamaño del alamcen, la cantidad de robots y la cantidad de cajas. Si mejoramos la cantidad de robots, el tiempo disminuiria sin embargo, los movimientos aumentarián, de la misma manera que si el almacen es más grande. 

- La manera eficiente, es mejorar el algoritmo de los robots cuando ya tengan una caja, para esto se pueden implementar algortimos de la distancia más corta para ir a un estante y colocar la caja y así disminuir sus movimientos, evitando a toda costa, que se muevan de manera aleatoria.

- Otra manera es, primero revisar todos sus alrededores y priorizar lo que haya encontrado, si hay una caja que la recoja y que no escoga moverse a una celda vacía, ya que esto disminuye el rendimiento y aumenta la cantidad de movimientos realiados por el robot.